# Import all necessary libraries

In [75]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import pandas as pd
from nltk.tokenize import word_tokenize
import re

# Load data

In [2]:
df = pd.read_csv("../data/allnews.csv", delimiter=',')
df.head()

,Unnamed: 0,tags,text,datetime
0,0,Новини Росії;Водний транспорт;Аварії та ДТП;На...,Затонув плавучий док ПД-50 Читайте також: За ...,2018-10-30 10:15
1,1,авіація;Жертви;Надзвичайні ситуації;Катастрофа...,"178 дорослих, троє дітей, шестеро членів екіпа...",2018-10-30 10:15
2,2,Новини України;Землетрус;Новини Одеси;Румунія;...,"Як Читайте також: За попередніми даними, пос...",2018-10-28 09:58
3,3,Новини України;авіація;Новини Одеси;Надзвичайн...,"Раніше 24 канал повідомляв, що увечері 27 жовт...",2018-10-27 22:47
4,4,Новини України;авіація;Новини Одеси;Надзвичайн...,"Як Читайте також: ""Всіх пасажирів евакуйовано...",2018-10-27 19:53


In [8]:
article_example = df['text'][0]
article_example

'Затонув плавучий док ПД-50 Читайте також:\xa0 За даними місцевого видання "СеверПост", під час аварії впало два баштових крани, один з яких завалився на авіаносець. Внаслідок аварії є постраждалі і можуть бути жертви. Четверо постраждалих, троє доставлені в медсанчастину, один –\xa0в міську лікарню, В одного з постраждалих поранення черевної порожнини і переохолодження, троє у стані середньої тяжкості – вони також зазнали переохолоджень. Водночас джерела видання повідомляють, що постраждалих може бути набагато більше.   Очевидці поширюють у мережі відео з місця надзвичайної ситуації. Дивіться відео про аварію під час ремонту авіаносця "Адмірал Кузнєцов" у Росії: Чим відомий\xa0" У Мурманській області Росії під час ремонту і модернізації єдиного російського авіносця "Адмірал Кузнєцов" сталась надзвичайна ситуація.'

# Generate clean sentences

1. Remove '\xa0' and 'Читайте також:'
2. Split sentences
3. Remove all non-numeric values
4. Tokenize sentences

In [76]:
def read_article(filename):
    """
    string -> list of strings
    """
    
    file = filename.replace(u'\xa0', u' ')
    file = file.replace('Читайте також:', '.')
    article = file.split(". ")
    
    sentences = []
    pattern = re.compile('[\W_]+')
    for sentence in article:
        sentences.append(word_tokenize(pattern.sub(' ', sentence)))
    return sentences

In [77]:
print(read_article(article_example))

[['Затонув', 'плавучий', 'док', 'ПД', '50'], ['За', 'даними', 'місцевого', 'видання', 'СеверПост', 'під', 'час', 'аварії', 'впало', 'два', 'баштових', 'крани', 'один', 'з', 'яких', 'завалився', 'на', 'авіаносець'], ['Внаслідок', 'аварії', 'є', 'постраждалі', 'і', 'можуть', 'бути', 'жертви'], ['Четверо', 'постраждалих', 'троє', 'доставлені', 'в', 'медсанчастину', 'один', 'в', 'міську', 'лікарню', 'В', 'одного', 'з', 'постраждалих', 'поранення', 'черевної', 'порожнини', 'і', 'переохолодження', 'троє', 'у', 'стані', 'середньої', 'тяжкості', 'вони', 'також', 'зазнали', 'переохолоджень'], ['Водночас', 'джерела', 'видання', 'повідомляють', 'що', 'постраждалих', 'може', 'бути', 'набагато', 'більше'], ['Очевидці', 'поширюють', 'у', 'мережі', 'відео', 'з', 'місця', 'надзвичайної', 'ситуації'], ['Дивіться', 'відео', 'про', 'аварію', 'під', 'час', 'ремонту', 'авіаносця', 'Адмірал', 'Кузнєцов', 'у', 'Росії', 'Чим', 'відомий', 'У', 'Мурманській', 'області', 'Росії', 'під', 'час', 'ремонту', 'і', 'м

# Find similarity matrix between sentences

In [92]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [93]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

# Generate summary

In [106]:
def generate_summary(filename, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    # Step 1 - Read text and tokenize
    sentences =  read_article(filename)
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    # Step 5 - Offcourse, output the summarize texr
    print("\nSummarize Text: \n", ". ".join(summarize_text))

In [108]:
generate_summary(article_example, 2)

Indexes of top ranked_sentence order are  [(0.19506544639257384, ['За', 'даними', 'місцевого', 'видання', 'СеверПост', 'під', 'час', 'аварії', 'впало', 'два', 'баштових', 'крани', 'один', 'з', 'яких', 'завалився', 'на', 'авіаносець']), (0.1817623792132247, ['Четверо', 'постраждалих', 'троє', 'доставлені', 'в', 'медсанчастину', 'один', 'в', 'міську', 'лікарню', 'В', 'одного', 'з', 'постраждалих', 'поранення', 'черевної', 'порожнини', 'і', 'переохолодження', 'троє', 'у', 'стані', 'середньої', 'тяжкості', 'вони', 'також', 'зазнали', 'переохолоджень']), (0.17885157127722226, ['Дивіться', 'відео', 'про', 'аварію', 'під', 'час', 'ремонту', 'авіаносця', 'Адмірал', 'Кузнєцов', 'у', 'Росії', 'Чим', 'відомий', 'У', 'Мурманській', 'області', 'Росії', 'під', 'час', 'ремонту', 'і', 'модернізації', 'єдиного', 'російського', 'авіносця', 'Адмірал', 'Кузнєцов', 'сталась', 'надзвичайна', 'ситуація']), (0.1478004356585752, ['Очевидці', 'поширюють', 'у', 'мережі', 'відео', 'з', 'місця', 'надзвичайної', 'с